# Mapping global indicators of spatial accessibility with regard to policy relevant thresholds for population health and wellbeing

Carl Higgs, Global Indicators project, 2020

This notebook draws on spatial estimates from the global indicators project to estimate percent of population living below, within and exceeding policy relevant threshold bounds for increased physical activity levels derived by Ester Cerin using the IPEN study populations.

Specifically, the analysis and mapping is concerned with threshold values for urban design and transport planning features associated with
  - (A) ≥80% probability of engaging in walking for transport and 
  - (B) reaching the WHO’s target of a ≥15% relative reduction in insufficient physical activity through walking
  
Thresholds are presented as 95% credible interval bounds, which population within 250m hexagonal grid segments across urban portions of the city are identified as being below, within or exceeding based on the estimates derived in the main project outputs.



## Import libraries and city specific parameters

In [1]:
import os
import numpy as np
import fiona
import pandas as pd
import geopandas as gpd
import argparse
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from textwrap import wrap
from matplotlib.backends.backend_pdf import PdfPages
import json
with open('../process/configuration/cities.json') as f:
  city_data = json.load(f)
exec(open('../process/data/GTFS/gtfs_config.py').read())

def valid_path(arg):
    arg = os.path.abspath(arg)
    if not os.path.exists(arg):
        msg = f"The path {arg} does not exist!"
        raise argparse.ArgumentTypeError(msg)
    else:
        return arg


import warnings
# filter out RuntimeWarnings, due to geopandas/fiona read file spam
# https://stackoverflow.com/questions/64995369/geopandas-warning-on-read-file
warnings.filterwarnings("ignore",category=RuntimeWarning)


## Import data

In [2]:
# Parse input arguments
# parser = argparse.ArgumentParser(description='Analyse processed results with regard to thresholds')
# parser.add_argument('-gpkg_cities',
#                     help='path to all cities summary results geopackage',
#                     default='./data/output/November 2020/global_indicators_city_2020-11-24.gpkg',
#                     type=valid_path)
# parser.add_argument('-gpkg_hexes',
#                     help='path to all cities hexagon grid results geopackage',
#                     default='./data/output/November 2020/global_indicators_hex_250m_2020-11-24.gpkg',
#                     type=valid_path)
# args = parser.parse_args()

# dummy parsing for interactive debugging
class Object(object):
    pass

args = Object()
args.gpkg_cities = os.path.abspath('../process/data/output/global_indicators_city_2021-06-21.gpkg')
args.gpkg_hexes = os.path.abspath('../process/data/output/global_indicators_hex_250m_2021-06-21.gpkg')

cities = gpd.read_file(args.gpkg_cities, layer='all_cities_combined')
cities.set_index('City',inplace=True)
# cities

In [3]:
hexes={}
for city in cities.index:
    hexes[city] = gpd.read_file(args.gpkg_hexes, layer=city.lower().replace(' ','_'))

In [4]:
hexes.keys()

dict_keys(['Maiduguri', 'Mexico City', 'Baltimore', 'Phoenix', 'Seattle', 'Sao Paulo', 'Hong Kong', 'Chennai', 'Bangkok', 'Hanoi', 'Adelaide', 'Melbourne', 'Sydney', 'Auckland', 'Graz', 'Ghent', 'Olomouc', 'Odense', 'Cologne', 'Lisbon', 'Barcelona', 'Valencia', 'Vic', 'Bern', 'Belfast'])

In [6]:
# Calculate public transport density for hexagons, required for one scenario
gtfs_analysis_date = '2021-06-16'
gtfs_gpkg = f'../process/data/GTFS/gtfs_frequent_transit_headway_{gtfs_analysis_date}_python.gpkg'

In [7]:
gtfs_gpkg

'../process/data/GTFS/gtfs_frequent_transit_headway_2021-06-16_python.gpkg'

In [11]:
points_in_polys = {}
point_hexes={}
for city in hexes.keys():
    _city_ = city.lower().replace(' ','_')
    if GTFS[_city_]==[]:
        transport_data = f"../process/data/output/{city_data['gpkgNames'][_city_]}"
        osm_layer = 'destinations'
        points_in_polys[city] = gpd.read_file(f"../process/data/output/{city_data['gpkgNames'][_city_]}",layer=osm_layer)
        points_in_polys[city] = points_in_polys[city].query('dest_name_full =="Public transport stop (any)"')
    else:
        if _city_ in dissolve_cities:
            gtfs_layer = f"{_city_}_stops_average_feeds_headway_{GTFS[_city_][-1]['start_date_mmdd']}_{GTFS[_city_][-1]['end_date_mmdd']}"
        else:
            gtfs_layer = f"{_city_}_stops_headway_{GTFS[_city_][-1]['start_date_mmdd']}_{GTFS[_city_][-1]['end_date_mmdd']}"
        points_in_polys[city] = gpd.read_file(gtfs_gpkg,layer=gtfs_layer)
        
    points_in_polys[city] = gpd.sjoin(points_in_polys[city],hexes[city],how='left', op='within')
    points_in_polys[city] = points_in_polys[city]['index_right'].dropna().astype(int)
    points_in_polys[city] = points_in_polys[city].reset_index().groupby('index_right').count().reset_index()
    points_in_polys[city].columns = ['index','pt_stops']
    point_hexes[city] = hexes[city].join(points_in_polys[city].set_index('index'),how='left').copy()
    point_hexes[city]['pt_stops_per_sqkm'] = point_hexes[city]['pt_stops']/hexes[city]['area_sqkm']
                                          

## Scientific colour mapping code

The below cell code uses the Breslow colour map from Fabio Crameri's (https://www.fabiocrameri.ch/colourmaps/)[Scientific colour maps, Version 7.0.0 (02.02.2021, scm-vn7.0)].  Specifically, the values from the file 'ScientificColourMaps7\batlow\batlow.txt' are used as text input.

Crameri, F. (2018). Scientific colour maps. Zenodo. http://doi.org/10.5281/zenodo.1243862

Crameri, F. (2018), Geodynamic diagnostics, scientific visualisation and StagLab 3.0, Geosci. Model Dev., 11, 2541-2562, doi:10.5194/gmd-11-2541-2018

Crameri, F., G.E. Shephard, and P.J. Heron (2020), The misuse of colour in science communication, Nature Communications, 11, 5444. doi:10.1038/s41467-020-19160-7

The Scientific color map values for the Breslow scale were used under MIT License terms, as below

Copyright (c) 2021 Fabio Crameri

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [31]:
from io import StringIO

batlow = StringIO("""0.005193 0.098238 0.349842
0.009065 0.104487 0.350933
0.012963 0.110779 0.351992
0.016530 0.116913 0.353070
0.019936 0.122985 0.354120
0.023189 0.129035 0.355182
0.026291 0.135044 0.356210
0.029245 0.140964 0.357239
0.032053 0.146774 0.358239
0.034853 0.152558 0.359233
0.037449 0.158313 0.360216
0.039845 0.163978 0.361187
0.042104 0.169557 0.362151
0.044069 0.175053 0.363084
0.045905 0.180460 0.364007
0.047665 0.185844 0.364915
0.049378 0.191076 0.365810
0.050795 0.196274 0.366684
0.052164 0.201323 0.367524
0.053471 0.206357 0.368370
0.054721 0.211234 0.369184
0.055928 0.216046 0.369974
0.057033 0.220754 0.370750
0.058032 0.225340 0.371509
0.059164 0.229842 0.372252
0.060167 0.234299 0.372978
0.061052 0.238625 0.373691
0.062060 0.242888 0.374386
0.063071 0.247085 0.375050
0.063982 0.251213 0.375709
0.064936 0.255264 0.376362
0.065903 0.259257 0.376987
0.066899 0.263188 0.377594
0.067921 0.267056 0.378191
0.069002 0.270922 0.378774
0.070001 0.274713 0.379342
0.071115 0.278497 0.379895
0.072192 0.282249 0.380434
0.073440 0.285942 0.380957
0.074595 0.289653 0.381452
0.075833 0.293321 0.381922
0.077136 0.296996 0.382376
0.078517 0.300622 0.382814
0.079984 0.304252 0.383224
0.081553 0.307858 0.383598
0.083082 0.311461 0.383936
0.084778 0.315043 0.384240
0.086503 0.318615 0.384506
0.088353 0.322167 0.384731
0.090281 0.325685 0.384910
0.092304 0.329220 0.385040
0.094462 0.332712 0.385116
0.096618 0.336161 0.385134
0.099015 0.339621 0.385090
0.101481 0.343036 0.384981
0.104078 0.346410 0.384801
0.106842 0.349774 0.384548
0.109695 0.353098 0.384217
0.112655 0.356391 0.383807
0.115748 0.359638 0.383310
0.118992 0.362849 0.382713
0.122320 0.366030 0.382026
0.125889 0.369160 0.381259
0.129519 0.372238 0.380378
0.133298 0.375282 0.379395
0.137212 0.378282 0.378315
0.141260 0.381240 0.377135
0.145432 0.384130 0.375840
0.149706 0.386975 0.374449
0.154073 0.389777 0.372934
0.158620 0.392531 0.371320
0.163246 0.395237 0.369609
0.167952 0.397889 0.367784
0.172788 0.400496 0.365867
0.177752 0.403041 0.363833
0.182732 0.405551 0.361714
0.187886 0.408003 0.359484
0.193050 0.410427 0.357177
0.198310 0.412798 0.354767
0.203676 0.415116 0.352253
0.209075 0.417412 0.349677
0.214555 0.419661 0.347019
0.220112 0.421864 0.344261
0.225707 0.424049 0.341459
0.231362 0.426197 0.338572
0.237075 0.428325 0.335634
0.242795 0.430418 0.332635
0.248617 0.432493 0.329571
0.254452 0.434529 0.326434
0.260320 0.436556 0.323285
0.266241 0.438555 0.320085
0.272168 0.440541 0.316831
0.278171 0.442524 0.313552
0.284175 0.444484 0.310243
0.290214 0.446420 0.306889
0.296294 0.448357 0.303509
0.302379 0.450282 0.300122
0.308517 0.452205 0.296721
0.314648 0.454107 0.293279
0.320834 0.456006 0.289841
0.327007 0.457900 0.286377
0.333235 0.459794 0.282937
0.339469 0.461685 0.279468
0.345703 0.463563 0.275998
0.351976 0.465440 0.272492
0.358277 0.467331 0.269037
0.364589 0.469213 0.265543
0.370922 0.471085 0.262064
0.377291 0.472952 0.258588
0.383675 0.474842 0.255131
0.390070 0.476711 0.251665
0.396505 0.478587 0.248212
0.402968 0.480466 0.244731
0.409455 0.482351 0.241314
0.415967 0.484225 0.237895
0.422507 0.486113 0.234493
0.429094 0.488011 0.231096
0.435714 0.489890 0.227728
0.442365 0.491795 0.224354
0.449052 0.493684 0.221074
0.455774 0.495585 0.217774
0.462539 0.497497 0.214518
0.469368 0.499393 0.211318
0.476221 0.501314 0.208148
0.483123 0.503216 0.205037
0.490081 0.505137 0.201976
0.497089 0.507058 0.198994
0.504153 0.508984 0.196118
0.511253 0.510898 0.193296
0.518425 0.512822 0.190566
0.525637 0.514746 0.187990
0.532907 0.516662 0.185497
0.540225 0.518584 0.183099
0.547599 0.520486 0.180884
0.555024 0.522391 0.178854
0.562506 0.524293 0.176964
0.570016 0.526186 0.175273
0.577582 0.528058 0.173775
0.585199 0.529927 0.172493
0.592846 0.531777 0.171449
0.600520 0.533605 0.170648
0.608240 0.535423 0.170104
0.615972 0.537231 0.169826
0.623739 0.539002 0.169814
0.631513 0.540752 0.170075
0.639301 0.542484 0.170622
0.647098 0.544183 0.171465
0.654889 0.545863 0.172603
0.662691 0.547503 0.174044
0.670477 0.549127 0.175747
0.678244 0.550712 0.177803
0.685995 0.552274 0.180056
0.693720 0.553797 0.182610
0.701421 0.555294 0.185478
0.709098 0.556772 0.188546
0.716731 0.558205 0.191851
0.724322 0.559628 0.195408
0.731878 0.561011 0.199174
0.739393 0.562386 0.203179
0.746850 0.563725 0.207375
0.754268 0.565033 0.211761
0.761629 0.566344 0.216322
0.768942 0.567630 0.221045
0.776208 0.568899 0.225930
0.783416 0.570162 0.230962
0.790568 0.571421 0.236160
0.797665 0.572682 0.241490
0.804709 0.573928 0.246955
0.811692 0.575187 0.252572
0.818610 0.576462 0.258303
0.825472 0.577725 0.264197
0.832272 0.579026 0.270211
0.838999 0.580339 0.276353
0.845657 0.581672 0.282631
0.852247 0.583037 0.289036
0.858747 0.584440 0.295572
0.865168 0.585882 0.302255
0.871505 0.587352 0.309112
0.877741 0.588873 0.316081
0.883878 0.590450 0.323195
0.889900 0.592087 0.330454
0.895809 0.593765 0.337865
0.901590 0.595507 0.345429
0.907242 0.597319 0.353142
0.912746 0.599191 0.360986
0.918103 0.601126 0.368999
0.923300 0.603137 0.377139
0.928323 0.605212 0.385404
0.933176 0.607369 0.393817
0.937850 0.609582 0.402345
0.942332 0.611867 0.411006
0.946612 0.614218 0.419767
0.950697 0.616649 0.428624
0.954574 0.619137 0.437582
0.958244 0.621671 0.446604
0.961696 0.624282 0.455702
0.964943 0.626934 0.464860
0.967983 0.629639 0.474057
0.970804 0.632394 0.483290
0.973424 0.635183 0.492547
0.975835 0.638012 0.501826
0.978052 0.640868 0.511090
0.980079 0.643752 0.520350
0.981918 0.646664 0.529602
0.983574 0.649590 0.538819
0.985066 0.652522 0.547998
0.986392 0.655470 0.557142
0.987567 0.658422 0.566226
0.988596 0.661378 0.575265
0.989496 0.664329 0.584246
0.990268 0.667280 0.593174
0.990926 0.670230 0.602031
0.991479 0.673165 0.610835
0.991935 0.676091 0.619575
0.992305 0.679007 0.628251
0.992595 0.681914 0.636869
0.992813 0.684815 0.645423
0.992967 0.687705 0.653934
0.993064 0.690579 0.662398
0.993111 0.693451 0.670810
0.993112 0.696314 0.679177
0.993074 0.699161 0.687519
0.993002 0.702006 0.695831
0.992900 0.704852 0.704114
0.992771 0.707689 0.712380
0.992619 0.710530 0.720639
0.992447 0.713366 0.728892
0.992258 0.716210 0.737146
0.992054 0.719049 0.745403
0.991837 0.721893 0.753673
0.991607 0.724754 0.761959
0.991367 0.727614 0.770270
0.991116 0.730489 0.778606
0.990855 0.733373 0.786976
0.990586 0.736265 0.795371
0.990307 0.739184 0.803810
0.990018 0.742102 0.812285
0.989720 0.745039 0.820804
0.989411 0.747997 0.829372
0.989089 0.750968 0.837979
0.988754 0.753949 0.846627
0.988406 0.756949 0.855332
0.988046 0.759964 0.864078
0.987672 0.762996 0.872864
0.987280 0.766047 0.881699
0.986868 0.769105 0.890573
0.986435 0.772184 0.899493
0.985980 0.775272 0.908448
0.985503 0.778378 0.917444
0.985002 0.781495 0.926468
0.984473 0.784624 0.935531
0.983913 0.787757 0.944626
0.983322 0.790905 0.953748
0.982703 0.794068 0.962895
0.982048 0.797228 0.972070
0.981354 0.800406 0.981267""")
cm_data = np.loadtxt(batlow)
from matplotlib.colors import LinearSegmentedColormap
batlow_map = LinearSegmentedColormap.from_list("batlow", cm_data[::-1])
batlow_map_4 = LinearSegmentedColormap.from_list("batlow", cm_data,4)
batlow_map_R = batlow_map.reversed()

## Scenario set up for threshold analyses

In [ ]:
# Analysis set up
scenarios={
  'A':'≥80% probability of engaging in walking for transport', 
  'B':'reaching the WHO’s target of a ≥15% relative reduction in insufficient physical activity through walking',
  'distances':'distances to destinations, measured up to a maximum distance target threshold of 500 metres'
}
scenario_style = {
    'A':{'colour':batlow_map(170),'line':'dashed','align':0.96},
    'B':{'colour':batlow_map(0),'line':'solid','align':0.93},
    'distances':{'colour':batlow_map(170),'line':'dashed','align':0.96},
    }
greq = '≥'
thresholds = {
'Mean 1000 m neighbourhood population per km²':{
  'data':'hexes', # the geopackage (hexes or points)
  'variable':'local_nh_population_density', # variable; a list is required if a function is specified
  'polarity':'positive', # which is better: more (positive)? or less (negative)?
  'scenarios':{
      'A':{
        'threshold':5665, # not used; we plot the interval
        'comparison':'>', # direction in which to evaluate success (e.g. is the aim to be greater than or less than the threshold?)
        'interval':(4790, 6750),
        'interval_type':'95% CrI'
        },
      'B':{
        'threshold':6491,
        'comparison':'>',
        'interval':(5677, 7823),
        'interval_type':'95% CrI' 
        }
  }
},
'Mean 1000 m neighbourhood street intersections per km²':{
  'data':'hexes',
  'variable':'local_nh_intersection_density',
  'polarity':'positive',
  'scenarios':{
      'A':{
        'threshold':98,
        'comparison':'>',
        'interval':(90, 110),
        'interval_type':'95% CrI'
        },
      'B':{
        'threshold':122,
        'comparison':'>',
        'interval':(106, 156),
        'interval_type':'95% CrI'
        }
  }
},
'Distance to nearest public transport stops (m; up to 500m)':{
  'data':'points',
  'layer':'samplePointsData',
  'point_function':min, # take the minimum of the OSM and GTFS pt data sources, axis=1 w/ fillna w/ np.nan
  'variable':['sp_nearest_node_pt_osm_any','sp_nearest_node_pt_gtfs_any'],
  'truncate_cutoff':500, # distance measures are only formally measured up to 500m, however truncation at 500 is required 
                         # for neatness when plotting continuous distribution due to full distance measurement method
  'polarity':'negative', # shorter distance is assumed to be better, so polarity is negative
  'scenarios':{
      'distances':{
        'threshold':400,
        'comparison':'<',
        'interval':(300,500),
        'interval_type':'distance (m)',
        # 'statistic':'pop_pct_access_500m_pt_any_binary',
        }
  }
},
'Distance to nearest park (m; up to 500m)':{
  'data':'points',
  'layer':'samplePointsData',
  'variable':'sp_nearest_node_public_open_space_any',
  'truncate_cutoff':500,
  'polarity':'negative',
  'scenarios':{
      'distances':{
        'threshold':400,
        'comparison':'<',
        'interval':(300,500),
        'interval_type':'distance (m)',
        # 'statistic':'pop_pct_access_500m_public_open_space_any_binary',
        }
  }
}}



## Analysis loop

In [ ]:

fontprops = fm.FontProperties(size=8)
# for city in ['Odense']:
for city in hexes.keys():
    print(city)
    study_region = cities.query(f'index=="{city}"').to_crs(hexes[city].crs).copy()
    bounds = study_region.bounds
    width = (bounds['maxx'].values[0]-bounds['minx'].values[0])
    height = (bounds['maxy'].values[0]-bounds['miny'].values[0])
    statistics = []
    # create a PdfPages object for file output
    if not os.path.exists('./reports'):
        os.mkdir('./reports')
    with PdfPages(f'reports/{city}_threshold_summary.pdf') as pdf:
        for indicator in thresholds.keys():
            data = thresholds[indicator]['data']
            variable = thresholds[indicator]['variable']
            indicator_scenarios = list(thresholds[indicator]['scenarios'].keys())
            polarity = thresholds[indicator]['polarity']
            # adjust colour scales for indicator polarities (more blue is better, or meeting achievements)
            if polarity == 'negative':
                cmap = batlow_map
                cmap_r = batlow_map_R
            else:
                cmap = batlow_map_R
                cmap_r = batlow_map
            # Aggregate point data (e.g. distances) to hexes
            if data == 'points':
                layer = thresholds[indicator]['layer']
                points = gpd.read_file(f"../process/data/output/{city_data['gpkgNames'][city.lower().replace(' ','_')]}",layer=layer)
                if 'point_function' in thresholds[indicator].keys():
                    points[''.join(variable)]=points[variable].apply(lambda x: thresholds[indicator]['point_function'](x.fillna(value=np.nan)),axis=1)
                    variable = ''.join(variable)
                if 'distances' in [s for s in scenarios.keys() if s in indicator_scenarios]:
                    # fix distances > 500m or NA to 650m, to facilitate classification and plotting of '> 500m' category
                    points[variable] = points[variable].mask(points[variable] > 500, 650).mask(points[variable].isna(), 650)
                    # ensure this hex variable doesn't exist, eg as a result of debugging code
                    point_hexes[city] = point_hexes[city][[c for c in point_hexes[city].columns if c!=variable]]
                    point_hexes[city][variable] = point_hexes[city].merge(points.groupby('hex_id')[variable].mean().reset_index(),
                                                              left_on='index', 
                                                              right_on='hex_id')[variable]
                data = 'hexes'
            # Process maps for indicators using the hex data
            if data == 'hexes':
                if 'hex_function' in thresholds[indicator].keys():
                    point_hexes[city][variable]=point_hexes[city][variable].apply(lambda x: thresholds[indicator]['hex_function'](x),axis=1)

                var_min = round(min(point_hexes[city][variable].dropna()),1)
                var_max = round(max(point_hexes[city][variable].dropna()),1)

                # map main indicator
                fig, ax = plt.subplots(1, 1, figsize=(11.69,8.27))
                ax.set_aspect('equal')
                study_region.plot(ax=ax, color='none', edgecolor='black',zorder=2)
                divider = make_axes_locatable(ax)
                cax = divider.append_axes("right", size="5%", pad=0.1)
                ax.set_xticks([])
                ax.set_yticks([])
                scalebar = AnchoredSizeBar(ax.transData,
                                           1000, '1000 m', 'lower right', 
                                           pad= .01,
                                           color='black',
                                           frameon=False,
                                           fontproperties=fontprops)
                ax.add_artist(scalebar)
                fig.suptitle("\n".join(wrap(indicator, 120 )))
                if 'distances' in indicator_scenarios:
                    point_hexes[city].query(f'{variable} < 500')\
                               .plot(column=variable, ax=ax, legend=True, cax=cax, cmap=cmap, zorder=1)
                else:
                    point_hexes[city].plot(column=variable, ax=ax, legend=True, cax=cax, cmap=cmap, zorder=1)
                ax.set_rasterized(True)
                pdf.savefig(fig,dpi=200)
                plt.clf()
                # map scenarios using custom splits
                interval_splits ={}
                splits = {}
                for scenario in [s for s in scenarios.keys() if s in indicator_scenarios]:
                    attributes = list(thresholds[indicator]['scenarios'][scenario].keys())
                    # categorical distribution plots for meeting scenarios
                    if ('interval' in attributes):
                        splits[scenario] = thresholds[indicator]['scenarios'][scenario]['interval']
                        interval_type = thresholds[indicator]['scenarios'][scenario]['interval_type']
                        if var_max in splits[scenario]:
                            splits[scenario] = [x if x!=var_max else var_max for x in splits[scenario]]
                        if var_min in splits[scenario]:
                            splits[scenario] = [x if x!=var_min else min(point_hexes[city][variable]) for x in splits[scenario]]
                        interval_splits[scenario] = list(splits[scenario]).copy()
                        split_labels = [f'within {interval_type} {splits[scenario]}']
                        if var_min < splits[scenario][0]:
                            splits[scenario] = [var_min]+list(splits[scenario])
                            split_labels = [f'below {interval_type} lower bound']+split_labels
                        if var_max > splits[scenario][-1]:
                            splits[scenario] = list(splits[scenario])+[var_max]
                            split_labels = split_labels+[f'exceeds {interval_type} upper bound']
                        #print(splits)
                        point_hexes[city][f'{variable}_{scenario}'] = pd.cut(point_hexes[city][variable], bins=splits[scenario], labels=[str(x) for x in range(0,len(split_labels))])
                        point_hexes[city][f'{variable}_{scenario}']
                        fig, ax = plt.subplots(figsize=(11.69,8.27))
                        ax.set_aspect('equal')
                        study_region.plot(ax=ax, color='none', edgecolor='black', zorder=2)
                        ax.set_xticks([])
                        ax.set_yticks([])
                        scalebar = AnchoredSizeBar(ax.transData,
                                                   1000, '1000 m', 'lower right', 
                                                   pad= .01,
                                                   color='black',
                                                   frameon=False,
                                                   fontproperties=fontprops)
                        ax.add_artist(scalebar)
                        fig.suptitle("\n".join(wrap(f'{scenario}: Estimated {indicator} requirement for {scenarios[scenario]}', 120 )))
                        if 'notes' in attributes:
                            ax.set_title(f"{thresholds[indicator]['scenarios'][scenario]['notes']}")
                        point_hexes[city].plot(column = f'{variable}_{scenario}',ax=ax,legend=True,cmap=cmap, zorder=1,legend_kwds={'borderaxespad':-4-height**.001, 'loc':'lower center'})
                        legend = ax.get_legend()
                        for text, label in zip(legend.get_texts(), split_labels):
                            text.set_text(label)
                        ax.set_rasterized(True)
                        pdf.savefig(fig, dpi=200)
                        plt.clf()
                    if ('statistic' in attributes):
                        statistics.append(thresholds[indicator]['scenarios'][scenario]['statistic'])
                    elif ('interval' in attributes):
                        # Estimated percentage of population meeting indicator threshold
                        percentages = (100*point_hexes[city]\
                                    .groupby([point_hexes[city][f'{variable}_{scenario}']])['pop_est']\
                                    .sum()\
                                    /point_hexes[city]['pop_est'].sum()).round(1)
                        for c in split_labels:
                            try:
                                statistic = f'pop_pct_{scenario} - {indicator} - {c}'
                                cities.loc[city,statistic] = percentages.loc[c]
                            except:
                                cities.loc[city,statistic] = 0
                            finally:
                                statistics.append(statistic)
                if scenario == 'distances':
                    # histogram of distances (including NaN as > 500m, along with other > 500m)
                    point_hexes[city][f'{variable}'].mask(point_hexes[city][variable] > 500, 650)\
                                              .mask(point_hexes[city][variable].isna(), 650)\
                                              .plot.hist(grid=False, bins = range(0, 700,50),xticks=range(0, 600,100),align='mid',color=batlow_map(255))
                    plt.text(618,0,">",verticalalignment='center')
                else:
                    point_hexes[city][f'{variable}'].hist(grid=False,color=batlow_map(255))
                plt.suptitle("\n".join(wrap(f'Histogram of {indicator}.',120)))
                              
                for scenario in [s for s in scenarios.keys() if s in indicator_scenarios]:
                    attributes = list(thresholds[indicator]['scenarios'][scenario].keys())
                    if ('interval' in attributes):
                        plt.text(0.5,scenario_style[scenario]["align"],f'\n\n{scenario}: {interval_type} {thresholds[indicator]["scenarios"][scenario]["interval"]}; {scenario_style[scenario]["line"]} range', color=scenario_style[scenario]["colour"], transform=plt.gcf().transFigure,ha='center',va='center')
                        for line in [x for x in splits[scenario][1:] if x!=var_max]:
                            plt.axvline(line, color='k', linestyle=scenario_style[scenario]["line"], linewidth=1)
                        plt.axvspan(*interval_splits[scenario], color=scenario_style[scenario]["colour"],alpha=0.6, zorder=2)
                plt.ylabel("Frequency")
                pdf.savefig(fig)
                plt.clf()
            
    plt.close('all')

cities[statistics].fillna(0).transpose().to_csv(f'./reports/Global Indicators 2020 - thresholds summary estimates.csv')
cities[statistics].fillna(0).transpose()

Maiduguri
Mexico City
Baltimore
Phoenix
Seattle
Sao Paulo
Hong Kong
Chennai
Bangkok
Hanoi
Adelaide
Melbourne
Sydney
Auckland
